In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [3]:
### data load
wine = pd.read_csv("http://youngho.iwinv.net/data/wine_data.csv")
wine.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
wine.shape

(178, 14)

In [5]:
### 입력변수(x)와 출력변수(y)
x = wine.iloc[:,1:]
#x.std = StandardScaler().fit_transform(x)
y = wine.Class

In [6]:
### Train & Test data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.4)

In [7]:
### [0,1]  조정
# x_train
x_train_min = x_train.min(axis=0)
x_train_max = (x_train - x_train_min).max(axis=0)
x_train = (x_train - x_train_min) / x_train_max
#x_Test
x_test = (x_test - x_train_min) / x_train_max

In [8]:
### MLP
mlp = MLPClassifier(solver = 'lbfgs',hidden_layer_sizes = (10,10))
model = mlp.fit(x_train,y_train)

In [9]:
###  예측
y_pred = model.predict(x_test)

In [10]:
### 교차표
pd.crosstab(y_test,y_pred)

col_0,1,2,3
Class,,,
1,27,0,0
2,1,25,0
3,3,1,15


## Solver, Alpha, Hidden layer sizes 조정

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
# Set the parameters by cross-validation
tuned_parameters= {'solver': ['lbfgs'],
                   'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
                   'hidden_layer_sizes' : [ (5,), (10,), (5,5), (10,10), (100,) ]}

In [18]:
grid=GridSearchCV(MLPClassifier(), tuned_parameters)
%time grid.fit(x_train, y_train)

C:\Users\CPB02GameN\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\CPB02GameN\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\CPB02GameN\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T

Wall time: 5.36 s


GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_gr

In [19]:
grid.best_params_

{'alpha': 0.001, 'hidden_layer_sizes': (10, 10), 'solver': 'lbfgs'}

In [20]:
### MLP
mlp=MLPClassifier(solver='lbfgs', alpha=grid.best_params_['alpha'],
                  hidden_layer_sizes=grid.best_params_['hidden_layer_sizes'])
model=mlp.fit(x_train, y_train)

In [21]:
###  예측
y_pred = model.predict(x_test)

In [22]:
### 교차표
pd.crosstab(y_test,y_pred)

col_0,1,2,3
Class,,,
1,27,0,0
2,1,25,0
3,0,0,19


## Classifier comparison

출처: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py (https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py)